# Lọc frame trùng từ videos

In [3]:
import cv2
import os
import numpy as np
from skimage.metrics import structural_similarity as ssim


In [4]:
input_dir = "./frames"        # thư mục chứa frame gốc
output_dir = "./filtered"     # nơi lưu frame đã lọc

os.makedirs(output_dir, exist_ok=True)


In [5]:
frames = sorted([f for f in os.listdir(input_dir) if f.endswith(".jpg")])


In [6]:
def calc_ssim(img1, img2):
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    score, _ = ssim(gray1, gray2, full=True)
    return score


In [7]:
THRESHOLD = 0.9   # độ tương đồng > 0.95 thì coi là giống nhau (bỏ)

prev_img = None
count = 0

for i, name in enumerate(frames):
    img_path = os.path.join(input_dir, name)
    img = cv2.imread(img_path)
    if img is None:
        continue

    if prev_img is None:
        # frame đầu tiên luôn giữ lại
        cv2.imwrite(os.path.join(output_dir, name), img)
        prev_img = img
        count += 1
        continue

    similarity = calc_ssim(prev_img, img)

    if similarity < THRESHOLD:
        # Giữ lại nếu khác đáng kể
        cv2.imwrite(os.path.join(output_dir, name), img)
        prev_img = img
        count += 1

print(f"Đã lọc xong: giữ lại {count} frame khác biệt đáng kể.")


Đã lọc xong: giữ lại 7 frame khác biệt đáng kể.


# Augmentation tự động để làm phong phú dữ liệu

In [3]:
import os
import cv2
import albumentations as A
from tqdm import tqdm

# --- Cấu hình thư mục ---
IMG_DIR = "./data_train/v2/images/train"
LABEL_DIR = "./data_train/v2/labels/train"
OUT_IMG_DIR = "./data_train/augmented/images"
OUT_LABEL_DIR = "./data_train/augmented/labels"

os.makedirs(OUT_IMG_DIR, exist_ok=True)
os.makedirs(OUT_LABEL_DIR, exist_ok=True)

# --- Định nghĩa augmentation ---
transform = A.Compose([
    A.RandomBrightnessContrast(p=0.6),
    A.HueSaturationValue(p=0.6),
    A.Blur(blur_limit=3, p=0.3),
    A.HorizontalFlip(p=0.5),
    A.Affine(rotate=(-10, 10), scale=(0.9, 1.1), p=0.6),
    A.Resize(640, 640)  # YOLO cần kích thước đồng nhất
],
    bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'])
)

# --- Hàm đọc nhãn YOLO ---
def read_yolo_label(label_path):
    boxes = []
    class_labels = []
    if not os.path.exists(label_path):
        return boxes, class_labels
    with open(label_path, "r") as f:
        for line in f.readlines():
            parts = line.strip().split()
            cls = int(parts[0])
            x, y, w, h = map(float, parts[1:])
            boxes.append([x, y, w, h])
            class_labels.append(cls)
    return boxes, class_labels

# --- Hàm ghi nhãn YOLO ---
def save_yolo_label(path, boxes, class_labels):
    with open(path, "w") as f:
        for cls, (x, y, w, h) in zip(class_labels, boxes):
            f.write(f"{int(cls)} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")

# --- Augment dữ liệu ---
for img_name in tqdm(os.listdir(IMG_DIR)):
    if not img_name.endswith(".jpg"):
        continue

    img_path = os.path.join(IMG_DIR, img_name)
    label_path = os.path.join(LABEL_DIR, img_name.replace(".jpg", ".txt"))

    image = cv2.imread(img_path)
    bboxes, class_labels = read_yolo_label(label_path)

    # Tạo nhiều bản augment cho mỗi ảnh
    for i in range(20):  # Tăng 5 lần mỗi ảnh
        transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)
        aug_img = transformed["image"]
        aug_bboxes = transformed["bboxes"]
        aug_labels = transformed["class_labels"]

        # Nếu không còn bbox hợp lệ thì bỏ qua
        if len(aug_bboxes) == 0:
            continue

        out_img_name = img_name.replace(".jpg", f"_aug_{i:03d}.jpg")
        out_label_name = img_name.replace(".jpg", f"_aug_{i:03d}.txt")

        cv2.imwrite(os.path.join(OUT_IMG_DIR, out_img_name), aug_img)
        save_yolo_label(os.path.join(OUT_LABEL_DIR, out_label_name), aug_bboxes, aug_labels)


100%|██████████| 9/9 [00:11<00:00,  1.27s/it]


In [2]:
import cv2
import os

# Đường dẫn thư mục ảnh và label
image_dir = "./data_train/augmented/images"
label_dir = "./data_train/augmented/labels"

# Lấy danh sách file ảnh
for image_file in os.listdir(image_dir):
    if not image_file.endswith(".jpg"):
        continue

    img_path = os.path.join(image_dir, image_file)
    label_path = os.path.join(label_dir, image_file.replace(".jpg", ".txt"))

    img = cv2.imread(img_path)
    h, w, _ = img.shape

    # Nếu có file label thì vẽ
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            for line in f:
                parts = line.strip().split()
                cls_id = int(parts[0])
                x_center, y_center, box_w, box_h = map(float, parts[1:])

                # Chuyển tọa độ YOLO → pixel
                x1 = int((x_center - box_w / 2) * w)
                y1 = int((y_center - box_h / 2) * h)
                x2 = int((x_center + box_w / 2) * w)
                y2 = int((y_center + box_h / 2) * h)

                # Vẽ hình chữ nhật
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(img,  str(cls_id), (x1, y1 - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    # Hiển thị
    cv2.imshow("Label Check", img)
    key = cv2.waitKey(0)
    if key == 27:  # ESC để thoát
        break

cv2.destroyAllWindows()
